# RAG Metrics Deep Dive

**Understanding How RAG Evaluation Metrics Work**


## The 6 Metrics We'll Explore

| # | Metric | Evaluates | Key Question |
|---|--------|-----------|-------------|
| 1 | **Faithfulness** | Generator | Is the answer grounded in context? |
| 2 | **Answer Relevancy** | Generator | Does the answer address the question? |
| 3 | **Context Precision** | Retriever | Are relevant chunks ranked at the top? |
| 4 | **Context Recall** | Retriever | Did we retrieve all necessary information? |
| 5 | **Context Entity Recall** | Retriever | Did we retrieve all important entities? |
| 6 | **Noise Sensitivity** | System | Does noise cause wrong answers? |

---

## Section 1: Setup & Environment

Let's set up our environment with all required imports and configure our LLM/Embedding models.

In [1]:
# Environment Setup
import os
import warnings
warnings.filterwarnings('ignore')

from dotenv import load_dotenv
load_dotenv()

# Verify API key
if os.getenv("OPENAI_API_KEY"):
    print("✅ OPENAI_API_KEY found")
else:
    print("❌ OPENAI_API_KEY not found - please set it in your .env file")

✅ OPENAI_API_KEY found


In [2]:
# Core Imports

# Standard library
import numpy as np
import pandas as pd
import asyncio
import json

# LangChain components
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [3]:
# Initialize LLM and Embeddings

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [4]:
# Helper function for running async code in Jupyter

def run_async(coro):
    """Helper to run async code in Jupyter notebooks"""
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            # We're in Jupyter with an existing loop
            import nest_asyncio
            nest_asyncio.apply()
            return loop.run_until_complete(coro)
        else:
            return asyncio.run(coro)
    except RuntimeError:
        return asyncio.run(coro)

print("✅ Async helper ready")

✅ Async helper ready


---

# Section 2: Faithfulness

**Faithfulness** checks if the generated answer *sticks to the facts* from the retrieved context. It detects **hallucinations** - when the LLM makes things up that aren't in the source material.


### 🔧 How It Works (3 Steps)

```
Step 1: Extract Claims --> Step 2: Verify each claim against Context --> Step 3: Calculate Faithfulness
```

### 📐 Formula

$$\text{Faithfulness} = \frac{\text{Number of claims supported by context}}{\text{Total number of claims}}$$

### Step 1: Extract Claims 

First, we use an LLM to break the answer into atomic, verifiable statements.

In [12]:
# 1. Setup the data
bot_answer = "You are allowed $200 per person, and alcohol is pre-approved."
retrieved_context = "Employees may spend up to $75 per person on client dinners. Receipts are required"

# 2. Define the extraction prompt 
claim_prompt = ChatPromptTemplate.from_template("""
Given the following response, extract ALL factual claims as a numbered list.
Each claim should be a single, verifiable statement.

Response: {response}
""")

# 3. Run Extraction
claims_chain = claim_prompt | llm | StrOutputParser()

claims = claims_chain.invoke({"response": bot_answer})
print("Claims:")
print(claims)

Claims:
1. You are allowed $200 per person.
2. Alcohol is pre-approved.


### Step 2: Verify Against Context 

Next, we check each atomic claim against the source text to detect hallucinations.

In [9]:
# Define the Verification Prompt
verification_prompt = ChatPromptTemplate.from_template("""
Given the following context and claim, determine if the claim is SUPPORTED by the context.
Answer "SUPPORTED" or "NOT SUPPORTED".

Context: {context}
Claim: {claim}
""")

verify_chain = verification_prompt | llm | StrOutputParser()

# List of extracted claims (from Step 1)
claim_list = [
    "You are allowed $200 per person.",
    "Alcohol is pre-approved."
]

# Run Verification
results = []
for claim in claim_list:
    verdict = verify_chain.invoke({
        "context": retrieved_context, 
        "claim": claim
    })

    if "NOT SUPPORTED" in verdict:
        score = 0
    elif "SUPPORTED" in verdict:
        score = 1
    else:
        score = 0 

    results.append(score)
    print(f"Claim: '{claim}' -> {verdict} (Score: {score})")


Claim: 'You are allowed $200 per person.' -> NOT SUPPORTED (Score: 0)
Claim: 'Alcohol is pre-approved.' -> NOT SUPPORTED (Score: 0)


### Step 3: Calculate the Score

In [10]:
# Manual faithfulness calculation

faithfulness_score = sum(results) / len(results)
print(f"Faithfulness Score: {faithfulness_score}")

Faithfulness Score: 0.0


---

## Section 3: Answer Relevancy Deep Dive

### What Answer Relevancy Measures

**Answer Relevancy** checks if the answer *actually answers* the question asked. It doesn't care if the answer is factually correct - just whether it's relevant to the question.


### 🔧 The "Reverse Engineering" Approach

Instead of directly comparing the answer to the question, we can:

1. **Generates hypothetical questions** from the answer ("What questions would this be a good answer to?")
2. **Compares embeddings** of generated questions with the original question
3. **Calculates similarity** - if the generated questions are similar to the original, the answer is relevant!

```
Generate Questions --> Embed All Questions --> Calculate Similarity   
```

### 📐 Formula

$$\text{Answer Relevancy} = \frac{1}{N} \sum_{i=1}^{N} \text{cosine\_similarity}(Hypothetical_{Question_i}, Original_{Question})$$

### Step 1: Reverse Engineering (Hypothetical Question Generation)

In [ ]:
# 1. Setup data
original_question = "What is the client dinner allowance?"
bot_answer = "Travel expenses must be submitted via Workday by Friday."

# 2. Define the Reverse Engineering Prompt
reverse_prompt = ChatPromptTemplate.from_template("""
Given the following answer, generate 3 distinct questions that this answer would be a good response to.
Answer: {answer}
Generate 3 Questions (one per line):
1. 
2. 
3. 
""")

# 3. Run Generation
gen_chain = reverse_prompt | llm | StrOutputParser()
generated_output = gen_chain.invoke({"answer": bot_answer})

print(f"Original: {original_question}")
print(f"Generated: \n{generated_output}")

Original: What is the client dinner allowance?
Generated: 
1. When is the deadline for submitting travel expenses?  
2. How should I submit my travel expenses for reimbursement?  
3. What platform do I need to use to report my travel expenses?  


### Step 2: Compare Embeddings

In [18]:
hypothetical_questions = [
    "When is the deadline for submitting travel expenses?",
    "How should I submit my travel expenses for reimbursement?",
    "What platform do I need to use to report my travel expenses?"
]

# Function to calculate Cosine Similarity
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

# 1. Get Embeddings for the Original Question
vec_original = embeddings.embed_query(original_question)

# 2. Loop through Hypothetical Questions and calculate similarity
scores = []
for q in hypothetical_questions:
    vec_hypo = embeddings.embed_query(q)
    similarity = cosine_similarity(vec_original, vec_hypo)
    scores.append(similarity)

### Step 3: Calculate Final Score

In [19]:
# 3. Average the scores
relevancy_score = sum(scores) / len(scores)

print(f"Individual Similarities: {scores}")
print(f"Final Answer Relevancy Score: {relevancy_score:.4f}")

Individual Similarities: [0.2912332475805235, 0.28654404773454767, 0.23218916334013615]
Final Answer Relevancy Score: 0.2700


---

## Section 4: Context Precision Deep Dive

### What Context Precision Measures

**Context Precision** evaluates whether the *most relevant chunks are ranked at the top* of your retrieval results. It's about **ranking quality**, not just whether you retrieved relevant information.


### 🔧 How It Works

1. For each retrieved chunk, determine if it's **relevant** to the question/reference
2. Calculate **Precision@K** at each position (weighted by position)
3. Relevant chunks at the **top** = higher score

### 📐 Formula

$$\text{Context Precision@K} = \frac{\sum_{k=1}^{K} (\text{Precision@k} \times \text{relevance}_k)}{\text{Total relevant items in top K}}$$

### Step 1: Relevance Verification (The Judge) 
We ask the LLM to judge each chunk individually against the Ground Truth.

In [23]:
# 1. Setup data
question = "What is the dinner allowance?"
ground_truth = "Employees may spend up to $75 on client dinners."
retrieved_chunks = [
    "Employees may spend up to $75 on client dinners.",
    "All travel expenses must be reported in Workday.",
    "The breakfast allowance is $20 per day.",
    "The company HQ is located in New York."
]

# 2. Define the Verification Prompt
# We ask the LLM to act as a binary judge
relevance_prompt = ChatPromptTemplate.from_template("""
Given the Question and Ground Truth, determine if the following Context Chunk is RELEVANT.

Question: {question}
Ground Truth: {ground_truth}
Context Chunk: {chunk}

Is this chunk relevant for answering the question? Answer only "RELEVANT" or "NOT RELEVANT".
""")

# 3. Run Verification Loop
relevance_chain = relevance_prompt | llm | StrOutputParser()

verdicts = []
print("🔍 Verifying Chunks...")

for chunk in retrieved_chunks:
    result = relevance_chain.invoke({
        "question": question,
        "ground_truth": ground_truth,
        "chunk": chunk
    })
    
    # Convert text response to binary (1 or 0)
    is_relevant = 1 if "RELEVANT" in result.upper() else 0
    verdicts.append(is_relevant)
    
    print(f"Chunk: '{chunk[:30]}...' -> {result.strip()} ({is_relevant})")

🔍 Verifying Chunks...
Chunk: 'Employees may spend up to $75 ...' -> RELEVANT (1)
Chunk: 'All travel expenses must be re...' -> NOT RELEVANT (1)
Chunk: 'The breakfast allowance is $20...' -> NOT RELEVANT (1)
Chunk: 'The company HQ is located in N...' -> NOT RELEVANT (1)


### Step 3 - Precision@K Calculation: Good Ranking

In [24]:
# Calculate Precision@K for our Travel Bot results
# Verdicts from Step 1: [1 (Rel), 1 (Rel), 0 (Irrel), 0 (Irrel)]
verdicts = [1, 1, 0, 0]

print("📊 Step 2: The Math (Precision@K Calculation)")
print("=" * 60)
print(f"Verdicts: {verdicts}")
print("\nCalculation:")

precisions = []
relevant_count = 0

for k, is_relevant in enumerate(verdicts, 1):
    if is_relevant:
        relevant_count += 1
    
    precision_at_k = relevant_count / k
    
    status = "✅ Relevant" if is_relevant else "❌ Irrelevant"
    # We only calculate/add precision if the item at k is relevant
    contribution = f"→ Adds {precision_at_k:.2f}" if is_relevant else "→ Skips"
    
    print(f"   Rank {k} ({status}): Precision@{k} = {relevant_count}/{k} = {precision_at_k:.2f} {contribution}")
    
    if is_relevant:
        precisions.append(precision_at_k)

total_relevant = sum(verdicts)
context_precision = sum(precisions) / total_relevant if total_relevant > 0 else 0

print(f"\n   Sum of precisions: {sum(precisions):.2f}")
print(f"   Total relevant items: {total_relevant}")
print(f"\n   📊 Final Context Precision: {context_precision:.2f}")

📊 Step 2: The Math (Precision@K Calculation)
Verdicts: [1, 1, 0, 0]

Calculation:
   Rank 1 (✅ Relevant): Precision@1 = 1/1 = 1.00 → Adds 1.00
   Rank 2 (✅ Relevant): Precision@2 = 2/2 = 1.00 → Adds 1.00
   Rank 3 (❌ Irrelevant): Precision@3 = 2/3 = 0.67 → Skips
   Rank 4 (❌ Irrelevant): Precision@4 = 2/4 = 0.50 → Skips

   Sum of precisions: 2.00
   Total relevant items: 2

   📊 Final Context Precision: 1.00


In [25]:
# Calculate Precision@K for BAD ranking (relevant at bottom)

# Bad ranking: [Not Relevant, Not Relevant, Relevant, Relevant]
bad_ranking = [False, False, True, True]

print("📊 BAD RANKING: Relevant chunks at BOTTOM")
print("=" * 60)
print("\nRanking: [❌ Not Rel, ❌ Not Rel, ✅ Relevant, ✅ Relevant]")
print("\nPrecision@K calculation:")

precisions_bad = []
relevant_count = 0
for k, is_relevant in enumerate(bad_ranking, 1):
    if is_relevant:
        relevant_count += 1
    precision_at_k = relevant_count / k
    contributes = "→ Contributes" if is_relevant else "→ Does NOT contribute"
    print(f"   Position {k}: Precision@{k} = {relevant_count}/{k} = {precision_at_k:.2f} {contributes}")
    if is_relevant:
        precisions_bad.append(precision_at_k)

total_relevant = sum(bad_ranking)
context_precision_bad = sum(precisions_bad) / total_relevant if total_relevant > 0 else 0

print(f"\n   Sum of contributing precisions: {sum(precisions_bad):.2f}")
print(f"   Total relevant items: {total_relevant}")
print(f"\n   📊 Context Precision (Bad Ranking): {context_precision_bad:.2f}")

📊 BAD RANKING: Relevant chunks at BOTTOM

Ranking: [❌ Not Rel, ❌ Not Rel, ✅ Relevant, ✅ Relevant]

Precision@K calculation:
   Position 1: Precision@1 = 0/1 = 0.00 → Does NOT contribute
   Position 2: Precision@2 = 0/2 = 0.00 → Does NOT contribute
   Position 3: Precision@3 = 1/3 = 0.33 → Contributes
   Position 4: Precision@4 = 2/4 = 0.50 → Contributes

   Sum of contributing precisions: 0.83
   Total relevant items: 2

   📊 Context Precision (Bad Ranking): 0.42


---

## Section 5: Context Recall Deep Dive

### What Context Recall Measures

**Context Recall** checks if you retrieved *all the necessary information* to answer the question. It measures **retrieval completeness**.


### 🔧 How It Works

1. Break down the **reference answer** into individual claims
2. Check if each claim can be **attributed** to the retrieved context
3. Calculate: claims found / total claims

### 📐 Formula

$$\text{Context Recall} = \frac{\text{Reference claims found in context}}{\text{Total claims in reference}}$$

### Step 1: Extract Ground Truth Claims 

First, we identify the specific facts we are looking for.

In [26]:
# 1. Setup data
ground_truth = "Employees may spend up to $75. Receipts are required for expenses over $25."
retrieved_context = [
    "Employees may spend up to $75 on client dinners.",
    "All expenses must be reported in Workday."
]

# (In Ragas, an LLM does this extraction. Here is the output:)
gt_claims = [
    "Employees may spend up to $75",           # Claim 1
    "Receipts are required for expenses over $25" # Claim 2
]

print(f"Target Claims to Find: {gt_claims}")

Target Claims to Find: ['Employees may spend up to $75', 'Receipts are required for expenses over $25']


### Step 2: Attribution Check (The Judge) 

We use the LLM to check if each Ground Truth claim exists inside the retrieved text. We use a specific prompt pattern to perform this "Attribution" check.

In [27]:
# Define Verification Prompt
attribution_prompt = ChatPromptTemplate.from_template("""
Can the following claim be attributed to (found in) the given context?
Context: {context}
Claim: {claim}

Answer "YES" if the claim is supported by the context, "NO" if it cannot be found.
""")

# Run Verification Loop
attribution_chain = attribution_prompt | llm | StrOutputParser()
attribution_results = []

print("🔍 Verifying Claims in Context...")

# Join context into one block for easier checking
full_context_text = "\n".join(retrieved_context)

for claim in gt_claims:
    result = attribution_chain.invoke({
        "context": full_context_text,
        "claim": claim
    })
    
    found = 1 if "YES" in result.upper() else 0
    attribution_results.append(found)
    
    status = "✅ Found" if found else "❌ MISSING"
    print(f"Claim: '{claim}' -> {result.strip()} ({status})")

🔍 Verifying Claims in Context...
Claim: 'Employees may spend up to $75' -> YES (✅ Found)
Claim: 'Receipts are required for expenses over $25' -> NO (❌ MISSING)


### Step 3: The Math 

Now we calculate the score based on our findings

In [28]:
# Total Ground Truth Claims: 2
# Claims Successfully Found: 1

context_recall = 1 / 2

print(f"Context Recall Score: {context_recall:.2f}")

Context Recall Score: 0.50


---

## Section 6: Context Entity Recall Deep Dive

### What Context Entity Recall Measures

**Context Entity Recall** checks if you retrieved context containing all the *important entities* (people, places, dates, organizations) mentioned in the reference answer.

### 📐 Formula

$$\text{Entity Recall} = \frac{\text{Entities in both reference AND context}}{\text{Total entities in reference}}$$

### Step 1: Extract Entities (The Judge) 
We ask the LLM to identify the "Must Have" entities from the Ground Truth.

In [ ]:
# 1. Setup data
ground_truth = "Expenses over $75 must be approved by Sarah Jenkins, the VP of Finance."
retrieved_chunks = [
    "Dinner expenses over $75 require approval from the VP of Finance.",
    "Sarah Jenkins handles all international travel requests."
]

# Compute the full text for comparison
retrieved_text = "\n".join(retrieved_chunks)

# 2. Define Entity Extraction Prompt
entity_prompt = ChatPromptTemplate.from_template("""
Extract all Named Entities (Person, Organization, Date, Location, Monetary Values, Roles) from the text.
Return a Python list of strings.

Text: {text}
Entities:
""")

# 3. Extract Entities from Ground Truth
extraction_chain = entity_prompt | llm | StrOutputParser()

# This asks the LLM to find the entities in the Ground Truth
gt_entities = extraction_chain.invoke({"text": ground_truth})

# Simulated result from the LLM:
print(f"Ground Truth Entities: {gt_entities}")
print(f"Retrieved Text Length: {len(retrieved_text)} chars")

Ground Truth Entities: ```python
entities = [
    "Sarah Jenkins",  # Person
    "VP of Finance",  # Role
    "$75"            # Monetary Value
]
```
Retrieved Text Length: 122 chars


### Step 2: Comparison Check 

Now we check if these specific strings appear in our retrieved_text.

In [33]:
# We compare the GT entities against the computed Retrieved Text
gt_entities = ["$75", "Sarah Jenkins", "VP of Finance"]
entities_found = []
context_lower = retrieved_text.lower()

print("🔍 Checking Entities...")
for entity in gt_entities:
    if entity.lower() in context_lower:
        entities_found.append(entity)
        print(f"   ✅ Found: '{entity}'")
    else:
        print(f"   ❌ MISSING: '{entity}'")

print(f"Entities Found List: {entities_found}")

🔍 Checking Entities...
   ✅ Found: '$75'
   ✅ Found: 'Sarah Jenkins'
   ✅ Found: 'VP of Finance'
Entities Found List: ['$75', 'Sarah Jenkins', 'VP of Finance']


### Step 3: The Math

In [34]:
# Total GT Entities: 3
# Found in Context: 3

entity_recall = 3 / 3

print(f"Entity Recall Score: {entity_recall:.2f}")

Entity Recall Score: 1.00


---

## Section 7: Noise Sensitivity Deep Dive

### What Noise Sensitivity Measures

**Noise Sensitivity** tests how much *irrelevant information* in the retrieved context causes errors in the answer. It measures **robustness** to noise.


### ⚠️ Important: Lower is Better!

Unlike other metrics where higher is better, for Noise Sensitivity:
- **0.0** = Great! Model ignores noise completely
- **1.0** = Bad! Model is very easily confused by irrelevant information

### 📐 Formula

$$\text{Noise Sensitivity} = \frac{\text{Incorrect claims from noisy context}}{\text{Total claims}}$$

### Step 1: Extract Claims from Bot Answer 
We break the answer down into atomic facts.

In [37]:
# 1. Setup data
bot_response = "The standard room rate is $250, but it was previously $150."
ground_truth = "The standard room rate is $250."

# 2. Define Extraction Prompt
claim_prompt = ChatPromptTemplate.from_template("""
Extract claims from the text. Return a list of strings.
Text: {text}
Claims:
""")

# 3. Run Extraction
claim_chain = claim_prompt | llm | StrOutputParser()

# Simulated Output from LLM
claims = claim_chain.invoke({"text": bot_response}) 

print(f"Extracted Claims:  \n{claims}")

Extracted Claims:  
1. The standard room rate is $250.
2. The previous room rate was $150.


### Step 2: Verification against Ground Truth 

We verify each claim against the Ground Truth (NOT the retrieved context). This is key. We want to know if the Bot is saying things outside the perfect answer.

In [38]:
claims = [
    "The standard room rate is $250",
    "It was previously $150"
]

# Define Verification Prompt
verification_prompt = ChatPromptTemplate.from_template("""
Is the following claim supported by the Ground Truth?
Ground Truth: {ground_truth}
Claim: {claim}

Answer "SUPPORTED" or "NOT SUPPORTED".
""")

verification_chain = verification_prompt | llm | StrOutputParser()

incorrect_claims = 0
total_claims = len(claims)

print("🔍 Verifying Claims against Ground Truth...")

for claim in claims:
    result = verification_chain.invoke({
        "ground_truth": ground_truth,
        "claim": claim
    })
    
    if "NOT SUPPORTED" in result:
        incorrect_claims += 1
        print(f"   Claim: '{claim}' -> ❌ NOT in GT (Noise Detected)")
    else:
        print(f"   Claim: '{claim}' -> ✅ Supported")


🔍 Verifying Claims against Ground Truth...
   Claim: 'The standard room rate is $250' -> ✅ Supported
   Claim: 'It was previously $150' -> ❌ NOT in GT (Noise Detected)


### Step 3: The Math

In [39]:
# Incorrect Claims (Noise): 1
# Total Claims: 2

noise_sensitivity = 1 / 2

print(f"Noise Sensitivity Score: {noise_sensitivity:.2f}")

Noise Sensitivity Score: 0.50


---

## 📚 Summary & Key Takeaways

### Quick Reference Table

| Metric | What It Measures | Calculation | Ideal |
|--------|-----------------|-------------|-------|
| **Faithfulness** | Hallucination detection | Supported claims / Total claims | 1.0 |
| **Answer Relevancy** | Answer addresses question | Avg cosine similarity of generated Qs | 1.0 |
| **Context Precision** | Ranking quality | Position-weighted precision | 1.0 |
| **Context Recall** | Retrieval completeness | Reference claims found / Total | 1.0 |
| **Entity Recall** | Entity coverage | Common entities / Reference entities | 1.0 |
| **Noise Sensitivity** | Robustness to noise | Incorrect claims / Total claims | 0.0 |

## Key Insights

1. **Faithfulness** and **Answer Relevancy** evaluate the **Generator (LLM)**
2. **Context Precision**, **Context Recall**, and **Entity Recall** evaluate the **Retriever**
3. **Noise Sensitivity** evaluates the **overall system's robustness**
4. Understanding the **intermediate steps** helps debug evaluation issues
5. Use these metrics **together** for comprehensive RAG evaluation
